In [13]:
import pandas as pd
import requests 
import json
from config import key
import gmaps
from census import Census

census_key = "85ac64b6b5a9c0901b00329d1ef41f0c53ccfc98"
c = Census(census_key,year=2013)

data = pd.read_csv('zip_bank_data.csv')

census_pd = c.acs5.get(('B23025_005E'),
                       {'for':'zip code tabulation area'})
census_pd = pd.DataFrame(census_pd)
census_pd = census_pd.rename(columns={'B23025_005E':'Unemployment',
                                     'zip code tabulation area':'Zipcode'})
census_pd.head()


,Unemployment,Zipcode
0,1464.0,00601
1,4767.0,00602
2,4341.0,00603
3,134.0,00606
4,1134.0,00610


In [18]:
data['Zipcode'] = pd.to_numeric(data['Zipcode'])
census_pd['Zipcode'] = pd.to_numeric(census_pd['Zipcode'])

data_full = data.merge(census_pd,on='Zipcode',how='inner')
data_full['Unemployment Rate'] = 100 * data_full['Unemployment']/data_full['Population']
data_full.head()



,Zipcode,Address,Population,Median Age,Household Income,Per Capita Income,Poverty Rate,Lat,Lng,Bank Count,Unemployment,Unemployment Rate
0,49058,"Hastings, MI 49058, USA",19031,40.8,46777.0,22137,13.183753,42.630692,-85.292938,9,1011.0,5.312385
1,71405,"Ball, LA 71405, USA",5687,35.8,55242.0,23941,15.737647,31.406180,-92.396174,12,260.0,4.571830
2,20634,"Great Mills, MD 20634, USA",6562,31.9,79944.0,35961,9.295946,38.220161,-76.496792,9,266.0,4.053642
3,38487,"Williamsport, TN 38487, USA",762,41.6,38125.0,18884,24.409449,35.731037,-87.241930,0,78.0,10.236220
4,58466,"Marion, ND 58466, USA",509,44.5,69844.0,36981,9.430255,46.559422,-98.348154,1,6.0,1.178782


In [39]:
fig = gmaps.figure()
location = data_full[['Lat','Lng']]
markers = gmaps.marker_layer(location)
fig.add_layer(gmaps.heatmap_layer(location, weights=data_full['Unemployment Rate'],
                                 max_intensity = 50,
                                  dissipating=False,point_radius=1))

fig

Figure(layout=FigureLayout(height='420px'))

In [41]:
bank_rate = data_full['Bank Count'].tolist()

fig = gmaps.figure()
layer = gmaps.symbol_layer(location,
                           fill_color='rgba(0,150,0,0.4)',
                           stroke_color='rgba(0,0,150,0.4)',scale=2,
                          info_box_content=[f'# of Banks In Area: {bank}'for bank in bank_rate])

fig.add_layer(layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [43]:
fig = gmaps.figure()

fig.add_layer(gmaps.heatmap_layer(location, weights=data_full['Unemployment Rate'],
                                 max_intensity = 50,
                                  dissipating=False,point_radius=1))
fig.add_layer(layer)

fig

Figure(layout=FigureLayout(height='420px'))